# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,2022-10-13 20:59:04,69.4541,-133.0374,26.60,86,100,6.91,overcast clouds
1,1,Troitskoye,RU,2022-10-13 20:59:05,52.9821,84.6757,36.21,77,10,6.55,clear sky
2,2,Cape Town,ZA,2022-10-13 20:59:06,-33.9258,18.4232,61.92,78,0,6.91,clear sky
3,3,Arraial Do Cabo,BR,2022-10-13 20:59:07,-22.9661,-42.0278,76.86,73,0,20.71,clear sky
4,4,Vestmannaeyjar,IS,2022-10-13 20:59:07,63.4427,-20.2734,41.67,65,100,7.56,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your vacation destination? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your vacation destination? "))

What is the minimum temperature you would like for your vacation destination? 20
What is the maximum temperature you would like for your vacation destination? 55


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,2022-10-13 20:59:04,69.4541,-133.0374,26.60,86,100,6.91,overcast clouds
1,1,Troitskoye,RU,2022-10-13 20:59:05,52.9821,84.6757,36.21,77,10,6.55,clear sky
4,4,Vestmannaeyjar,IS,2022-10-13 20:59:07,63.4427,-20.2734,41.67,65,100,7.56,overcast clouds
5,5,Busselton,AU,2022-10-13 20:59:08,-33.6500,115.3333,53.46,64,96,1.39,overcast clouds
6,6,Tarbagatay,RU,2022-10-13 20:59:09,51.4803,107.3602,36.93,75,79,10.78,broken clouds
7,7,Oktyabrskiy,RU,2022-10-13 20:59:10,54.4815,53.4710,48.63,82,100,2.48,overcast clouds
9,9,Geraldton,AU,2022-10-13 20:59:11,-28.7667,114.6000,41.41,87,0,5.75,clear sky
18,18,Punta Arenas,CL,2022-10-13 20:59:18,-53.1500,-70.9167,50.11,46,40,20.71,scattered clouds
20,20,Launceston,AU,2022-10-13 20:59:20,-41.4500,147.1667,49.64,90,65,16.11,broken clouds
23,23,The Pas,CA,2022-10-13 20:59:21,53.8251,-101.2541,39.22,60,100,26.46,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
preferred_cities_df.count()


City_ID                222
City                   222
Country                222
Date                   222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,2022-10-13 20:59:04,69.4541,-133.0374,26.60,86,100,6.91,overcast clouds
1,1,Troitskoye,RU,2022-10-13 20:59:05,52.9821,84.6757,36.21,77,10,6.55,clear sky
4,4,Vestmannaeyjar,IS,2022-10-13 20:59:07,63.4427,-20.2734,41.67,65,100,7.56,overcast clouds
5,5,Busselton,AU,2022-10-13 20:59:08,-33.6500,115.3333,53.46,64,96,1.39,overcast clouds
6,6,Tarbagatay,RU,2022-10-13 20:59:09,51.4803,107.3602,36.93,75,79,10.78,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
670,670,Gravdal,NO,2022-10-13 21:03:13,68.1214,13.5031,40.89,87,75,2.30,broken clouds
676,676,Ulaanbaatar,MN,2022-10-13 21:02:54,47.9077,106.8832,30.92,71,2,5.48,clear sky
677,677,Shelburne,CA,2022-10-13 21:06:09,44.0787,-80.2041,52.90,72,86,15.30,overcast clouds
683,683,Waipawa,NZ,2022-10-13 21:06:12,-41.4122,175.5153,46.96,72,94,8.12,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df =  preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
0,Tuktoyaktuk,CA,26.60,overcast clouds,69.4541,-133.0374
1,Troitskoye,RU,36.21,clear sky,52.9821,84.6757
4,Vestmannaeyjar,IS,41.67,overcast clouds,63.4427,-20.2734
5,Busselton,AU,53.46,overcast clouds,-33.6500,115.3333
6,Tarbagatay,RU,36.93,broken clouds,51.4803,107.3602
7,Oktyabrskiy,RU,48.63,overcast clouds,54.4815,53.4710
9,Geraldton,AU,41.41,clear sky,-28.7667,114.6000
18,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167
20,Launceston,AU,49.64,broken clouds,-41.4500,147.1667
23,The Pas,CA,39.22,overcast clouds,53.8251,-101.2541


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,26.60,overcast clouds,69.4541,-133.0374,
1,Troitskoye,RU,36.21,clear sky,52.9821,84.6757,
4,Vestmannaeyjar,IS,41.67,overcast clouds,63.4427,-20.2734,
5,Busselton,AU,53.46,overcast clouds,-33.6500,115.3333,
6,Tarbagatay,RU,36.93,broken clouds,51.4803,107.3602,
...,...,...,...,...,...,...,...
670,Gravdal,NO,40.89,broken clouds,68.1214,13.5031,
676,Ulaanbaatar,MN,30.92,clear sky,47.9077,106.8832,
677,Shelburne,CA,52.90,overcast clouds,44.0787,-80.2041,
683,Waipawa,NZ,46.96,overcast clouds,-41.4122,175.5153,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,26.60,overcast clouds,69.4541,-133.0374,End of The Road Ltd.
1,Troitskoye,RU,36.21,clear sky,52.9821,84.6757,
4,Vestmannaeyjar,IS,41.67,overcast clouds,63.4427,-20.2734,Glamping & Camping
5,Busselton,AU,53.46,overcast clouds,-33.6500,115.3333,Observatory Guest House
6,Tarbagatay,RU,36.93,broken clouds,51.4803,107.3602,"Meditativnyy Tsentr ""Sansara"""
7,Oktyabrskiy,RU,48.63,overcast clouds,54.4815,53.4710,Гостиница Октябрьская
9,Geraldton,AU,41.41,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
18,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
20,Launceston,AU,49.64,broken clouds,-41.4500,147.1667,Hotel Grand Chancellor Launceston
23,The Pas,CA,39.22,overcast clouds,53.8251,-101.2541,Wescana Inn


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [29]:
hotel_df.count()

City                   222
Country                222
Max Temp               222
Current Description    222
Lat                    222
Lng                    222
Hotel Name             222
dtype: int64

In [38]:
import numpy as np
# Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('',np.nan,inplace=True)
clean_hotel_df=hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,26.60,overcast clouds,69.4541,-133.0374,End of The Road Ltd.
4,Vestmannaeyjar,IS,41.67,overcast clouds,63.4427,-20.2734,Glamping & Camping
5,Busselton,AU,53.46,overcast clouds,-33.6500,115.3333,Observatory Guest House
6,Tarbagatay,RU,36.93,broken clouds,51.4803,107.3602,"Meditativnyy Tsentr ""Sansara"""
7,Oktyabrskiy,RU,48.63,overcast clouds,54.4815,53.4710,Гостиница Октябрьская
9,Geraldton,AU,41.41,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
18,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
20,Launceston,AU,49.64,broken clouds,-41.4500,147.1667,Hotel Grand Chancellor Launceston
23,The Pas,CA,39.22,overcast clouds,53.8251,-101.2541,Wescana Inn
24,Dalvik,IS,37.33,overcast clouds,65.9702,-18.5286,Dalvík Hostel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [43]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [40]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [48]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>End of The Road Ltd.</dd>\n<dt>City</dt><dd>Tuktoyaktuk</dd>\n<dt>Country</dt><dd>CA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 26.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Glamping & Camping</dd>\n<dt>City</dt><dd>Vestmannaeyjar</dd>\n<dt>Country</dt><dd>IS</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 41.67 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Observatory Guest House</dd>\n<dt>City</dt><dd>Busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 53.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Meditativnyy Tsentr "Sansara"</dd>\n<dt>City</dt><dd>Tarbagatay</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>broken clouds and 36.93 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Гостиница Октябрьская</dd>\n<dt>City</dt><dd>Oktyabrskiy</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 48.63 °F</dd>\n</dl>\n',
 '\n<dl

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [49]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,69.4541,-133.0374
4,63.4427,-20.2734
5,-33.6500,115.3333
6,51.4803,107.3602
7,54.4815,53.4710
9,-28.7667,114.6000
18,-53.1500,-70.9167
20,-41.4500,147.1667
23,53.8251,-101.2541
24,65.9702,-18.5286


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [50]:
# Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))